In [4]:
import argparse
import os
import sys
import numpy as np
import tensorrt as trt
import cv2
import torchvision
import torch
import time
import traceback
import importlib
import common_dynamic as common
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

torch.cuda.init()

import argparse
import os
import sys
import numpy as np
import tensorrt as trt
import cv2
import torchvision
import torch
import time
import traceback
import importlib
import common_dynamic as common
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

# torch.cuda.init()


class PEOPLE_DETECTOR:

    def __init__(self):  
#         self.logger = logger

        
        self.num_classes = list(range(3))
        self.threshold = 0.7
        self.nmsthre = 0.5


        self.input_h = 544
        self.input_w = 960

        self.stride = 16
        self.box_norm = 35.0
        self.grid_h = 34
        self.grid_w = 60
        
        self.grid_calculator()

    def load(self, weights):
        self.weights = weights
        self.Engine = common.Engine()      
        self.Engine.make_context(self.weights)
        self.batchsize = 1

    def parse_input(self,input_data_batch):
        res = []
        for input_data in input_data_batch:
            frame = input_data['framedata']['frame']
            bbox = input_data['bbox']
            cropped_img = common.getCropByFrame(frame,bbox)
            res.append(cropped_img)
        return res
    
    
    def grid_calculator(self):

        norm_data_x = torch.zeros([self.grid_h, self.grid_w], dtype=torch.float32, device=torch.device("cuda"))
        norm_data_y = torch.zeros([self.grid_h, self.grid_w], dtype=torch.float32, device=torch.device("cuda"))

        self.grid_h = int(self.input_h / self.stride)
        self.grid_w = int(self.input_w / self.stride)
        self.grid_size = self.grid_h * self.grid_w

        for i in range(self.grid_h):
            value = (i * self.stride + 0.5) / self.box_norm
            norm_data_y[i,:] = value

        for i in range(self.grid_w):
            value = (i * self.stride + 0.5) / self.box_norm
            norm_data_x[:,i] = value
#         self.norm_data_x = norm_data_x
#         self.norm_data_y = norm_data_y    
        self.norm_data_x = torch.stack((norm_data_x,norm_data_x,norm_data_x))
        self.norm_data_y = torch.stack((norm_data_y,norm_data_y,norm_data_y))

    


    def postprocess(self,outputs, scale_list):
        """
        Postprocesses the inference output
        Args:
            outputs (list of float): inference output
            min_confidence (float): min confidence to accept detection
            analysis_classes (list of int): indices of the classes to consider

        Returns: list of list tuple: each element is a two list tuple (x, y) representing the corners of a bb
        """
        output_list = []
        ori_h = scale_list[0][0]
        ori_w = scale_list[0][1]

        tensor_x1 = torch.stack((outputs[0][0],outputs[0][4],outputs[0][8]))
        tensor_y1 = torch.stack((outputs[0][1],outputs[0][5],outputs[0][9]))
        tensor_x2 = torch.stack((outputs[0][2],outputs[0][6],outputs[0][10]))
        tensor_y2 = torch.stack((outputs[0][3],outputs[0][7],outputs[0][11]))

#         tensor_x1 = torch.cat([outputs[0][0],outputs[0][4],outputs[0][8]],0).view(-1,self.grid_h,self.grid_w)
#         tensor_y1 = torch.cat([outputs[0][1],outputs[0][5],outputs[0][9]],0).view(-1,self.grid_h,self.grid_w)
#         tensor_x2 = torch.cat([outputs[0][2],outputs[0][6],outputs[0][10]],0).view(-1,self.grid_h,self.grid_w)
#         tensor_y2 = torch.cat([outputs[0][3],outputs[0][7],outputs[0][11]],0).view(-1,self.grid_h,self.grid_w)
        tensor_x1 = (tensor_x1 - self.norm_data_x) * -35
        tensor_y1 = (tensor_y1 - self.norm_data_y) * -35
        tensor_x2 = (tensor_x2 + self.norm_data_x) * 35
        tensor_y2 = (tensor_y2 + self.norm_data_y) * 35

        
        score_tensor = outputs[1] >= self.threshold
        
        for label in self.num_classes : 
#             ii = label * 4
            
#             tensor_x1 = (outputs[0][ii] - self.norm_data_x) * -35
#             tensor_y1 = (outputs[0][ii + 1] - self.norm_data_y) * -35
#             tensor_x2 = (outputs[0][ii + 2] + self.norm_data_x) * 35
#             tensor_y2 = (outputs[0][ii + 3] + self.norm_data_y) * 35

#             new_x1 = tensor_x1[score_tensor[label]]
#             new_y1 = tensor_y1[score_tensor[label]]
#             new_x2 = tensor_x2[score_tensor[label]]
#             new_y2 = tensor_y2[score_tensor[label]]

            new_x1 = tensor_x1[label][score_tensor[label]]
            new_y1 = tensor_y1[label][score_tensor[label]]
            new_x2 = tensor_x2[label][score_tensor[label]]
            new_y2 = tensor_y2[label][score_tensor[label]]


            box = torch.stack((new_x1,new_y1,new_x2,new_y2),1)
            score = outputs[1][label][score_tensor[label]]

            nms_out_index = torchvision.ops.nms(
                box,
                score,
                self.nmsthre,
            )
            output_list.append(None)
            
            ############################################
            result_box = box[nms_out_index].detach().cpu().numpy()
            result_score = score[nms_out_index].detach().cpu().numpy()

            for idx, bbox in enumerate(result_box):
                if isinstance(bbox, type(None)):
                    output_list.append(None)
                    continue
                x1, y1, x2, y2 = map(int, bbox)
                x1 = int((x1 * ori_w)/self.input_w)
                y1 = int((y1 * ori_h)/self.input_h)
                x2 = int((x2 * ori_w)/self.input_w)
                y2 = int((y2 * ori_h)/self.input_h)
                score = str(result_score[idx])
                out = {"bbox":[x1, y1, x2, y2], "score":score, "label":label}
                output_list.append(out)
            ############################################

        return output_list
        


    def preprocess(self,frame_batch) : 

#         input_data = torch.zeros([3, self.input_h, self.input_w], dtype=torch.float32, device=torch.device("cuda")).fill_(144)
        scale_list = []
        
        for idx, frame in enumerate(frame_batch) :
            _, h, w = frame.shape
            permute = [2, 1, 0]
            frame = frame[permute,:,:]
            resized_img = torchvision.transforms.functional.resize(frame, (self.input_h, self.input_w)).float()
            input_data = resized_img.div(255.0)
#             input_data[:,:self.input_h,:self.input_w] = resized_img 
            scale_list.append([h,w])
        return input_data, scale_list

    
    def inference(self,input_data) : 
        output_data = self.Engine.do_inference_v2(input_data)
        return output_data

    
    def parse_output(self,input_data_batch,output_batch,reference_CM):
        res = []
        for idx_i, data in enumerate(input_data_batch): 
            framedata = data['framedata']
            scenario = data['scenario']
            channel_id = framedata['meta']['source']['channel_id']
            if output_batch == None:
                input_data = dict()
                input_data["framedata"] = framedata
                input_data["bbox"] = None
                input_data["scenario"] = scenario   
                input_data["data"] = None
                input_data["available"] = False
                res.append(input_data)
                continue
            for idx_j, output in enumerate(output_batch): 

                if isinstance(output, type(None)):
                    input_data = dict()
                    input_data["framedata"] = framedata
                    input_data["bbox"] = None
                    input_data["scenario"] = scenario   
                    input_data["data"] = None   
                    input_data["available"] = False
                    res.append(input_data)
                    continue
                    
                label = int(output['label']) 
                frame_count = framedata['meta']['source']['frame_count']

                if int(label) != 0 : 
                    input_data = dict()
                    input_data["framedata"] = framedata
                    input_data["bbox"] = None
                    input_data["scenario"] = scenario   
                    input_data["data"] = None   
                    input_data["available"] = False
                    res.append(input_data)
                    continue

                input_data = dict()
                input_data["framedata"] = framedata
                input_data["bbox"] = output['bbox']
                input_data["scenario"] = scenario   
                input_data["data"] = {"score":output['score'], "label":str(output['label'])}
                input_data["available"] = True
                res.append(input_data)
        return res  
        
    def run_inference(self, input_data_batch, unavailable_routing_data_batch, reference_CM=None):

        t1 = time.time()

        parsed_input_batch = self.parse_input(input_data_batch)
        t2 = time.time()
        
        x,scale_list = self.preprocess(parsed_input_batch)
        t3 = time.time()
        
        x = self.inference(x)
        t4 = time.time()
        
        output_batch = self.postprocess(x,scale_list)
        t5 = time.time()
        

        output = self.parse_output(input_data_batch,output_batch,reference_CM)
        t6 = time.time()
        frame_time = (t6 - t1) / len(input_data_batch)
        
#         print(f'[PEOPLE_DETECTOR] 1.parse_input : {t2 - t1}, 2.preprocess : {t3 - t2}, 3.inference : {t4 - t3}, 4.postprocess : {t5 - t4}, 5.parse_output : {t6 - t5},  6.total : {t6 - t1} 7. per_frame_time : {frame_time} 8. input_data_size : {len(input_data_batch)}')
#         print(f'________________________________________________________________________')
        
        return output, unavailable_routing_data_batch


    
weights = '/DATA_17/ij/test/best_model_people.trt'

img_path = '/DATA_17/ij/test/test_image.jpeg'
# image = cv2.imread(img_path)[..., ::-1]#BGR 순서를 RGB로 뒤집습니다.
image = cv2.imread(img_path)
frame = np.copy(image)

infe_frame = torch.from_numpy(frame).to(torch.device("cuda"))
infe_frame = infe_frame.permute(2, 0, 1) 


pe = PEOPLE_DETECTOR()
pe.load(weights)


input_data, scale_list = pe.preprocess([infe_frame])

result = pe.inference(input_data)
# print(f'result : {result[0].shape} {result[1].shape}')#result : torch.Size([12, 34, 60]) torch.Size([3, 34, 60])

# output_batch = pe.postprocess(result, scale_list)

# print(output_batch)

# image = cv2.imread(img_path)


# for result in output_batch:
#     xmin, ymin, xmax, ymax = result['bbox']
#     color = [255, 0, 0] 
#     cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 2)
# plt.imshow(image)
# plt.show()




[TensorRT] WARNING: The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. TensorRT maintains only a single logger pointer at any given time, so the existing value, which can be retrieved with getLogger(), will be used instead. In order to use a new logger, first destroy all existing builder, runner or refitter objects.

[TensorRT] WARNING: Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0
[TensorRT] WARNING: TensorRT was linked against cuDNN 8.2.1 but loaded cuDNN 8.2.0


In [ ]:

input_h = 544
input_w = 960
stride = 16
box_norm = 35

grid_h = 34
grid_w = 60
norm_data_x = torch.zeros([grid_h, grid_w], dtype=torch.float32, device=torch.device("cuda"))
norm_data_y = torch.zeros([grid_h, grid_w], dtype=torch.float32, device=torch.device("cuda"))


grid_h = int(input_h / stride)
grid_w = int(input_w / stride)
grid_size = grid_h * grid_w

for i in range(grid_h):
    value = (i * stride + 0.5) / box_norm
    norm_data_y[i,:] = value


for i in range(grid_w):
    value = (i * stride + 0.5) / box_norm
    norm_data_x[:,i] = value

# norm_data_x = torch.stack((norm_data_x,norm_data_x,norm_data_x))
# norm_data_y = torch.stack((norm_data_y,norm_data_y,norm_data_y))


In [50]:
tensor_x1 = torch.stack((result[0][0],result[0][4],result[0][8]))
tensor_y1 = torch.stack((result[0][1],result[0][5],result[0][9]))
tensor_x2 = torch.stack((result[0][2],result[0][6],result[0][10]))
tensor_y2 = torch.stack((result[0][3],result[0][7],result[0][11]))

tensor_x1 = (tensor_x1 - norm_data_x) * -35
tensor_y1 = (tensor_y1 - norm_data_y) * -35
tensor_x2 = (tensor_x2 + norm_data_x) * 35
tensor_y2 = (tensor_y2 + norm_data_y) * 35



tensor([[[-3.7571e+01, -4.2035e+01, -6.6948e+01,  ...,  8.1019e+02,
           8.6032e+02,  9.4193e+02],
         [-4.5277e+01, -8.1832e+01, -1.5286e+02,  ...,  7.6436e+02,
           8.6874e+02,  9.4603e+02],
         [-1.3719e+01, -5.0880e+01, -8.6292e+01,  ...,  8.3353e+02,
           9.0191e+02,  9.4676e+02],
         ...,
         [-3.0834e+01, -8.6500e-01,  1.2328e+01,  ...,  9.0063e+02,
           9.1344e+02,  9.4133e+02],
         [-6.0671e+00,  4.2042e+00,  1.6960e+01,  ...,  9.0376e+02,
           9.1462e+02,  9.4244e+02],
         [-1.1248e+01,  5.2154e+00,  1.2388e+01,  ...,  9.0657e+02,
           9.1830e+02,  9.4237e+02]],

        [[-1.6456e+01, -1.9195e+01, -5.2801e+01,  ...,  8.4705e+02,
           8.7772e+02,  9.4952e+02],
         [-2.6306e+01, -7.4261e+01, -1.0313e+02,  ...,  8.5062e+02,
           8.8776e+02,  9.4362e+02],
         [-3.5102e+01, -6.6522e+01, -6.8397e+01,  ...,  8.5851e+02,
           8.6973e+02,  9.3407e+02],
         ...,
         [-1.3648e+01, -6

In [16]:

s = time.time()
for i in range(1000) : 
    tensor_x1 = torch.cat([result[0][0],result[0][4],result[0][8]],0).view(-1,grid_h,grid_w)
    tensor_y1 = torch.cat([result[0][1],result[0][5],result[0][9]],0).view(-1,grid_h,grid_w)
    tensor_x2 = torch.cat([result[0][2],result[0][6],result[0][10]],0).view(-1,grid_h,grid_w)
    tensor_y2 = torch.cat([result[0][3],result[0][7],result[0][11]],0).view(-1,grid_h,grid_w)
    tensor_x1 = (tensor_x1 - norm_data_x) * -35
    tensor_y1 = (tensor_y1 - norm_data_y) * -35
    tensor_x2 = (tensor_x2 + norm_data_x) * 35
    tensor_y2 = (tensor_y2 + norm_data_y) * 35

print(time.time()-s)

0.19188308715820312


In [18]:
s = time.time()
for i in range(1000) : 

    tensor_x1 = (result[0][0] - norm_data_x) * -35
    tensor_x2 = (result[0][4] - norm_data_x) * -35
    tensor_x3 = (result[0][8] - norm_data_x) * -35

    
    tensor_y1 = (result[0][1] - norm_data_y) * -35
    tensor_y2 = (result[0][5] - norm_data_y) * -35
    tensor_y3 = (result[0][9] - norm_data_y) * -35

                
    tensor_x21 = (result[0][2] + norm_data_x) * 35
    tensor_x22 = (result[0][6] + norm_data_x) * 35
    tensor_x23 = (result[0][10] + norm_data_x) * 35
    
    tensor_y21 = (result[0][3] + norm_data_y) * 35
    tensor_y22 = (result[0][7] + norm_data_y) * 35
    tensor_y23 = (result[0][11] + norm_data_y) * 35

print(time.time()-s)

0.35172414779663086


In [49]:
s = time.time()
for i in range(1000) : 
    tensor_x1 = torch.stack((result[0][0],result[0][4],result[0][8]))
    tensor_y1 = torch.stack((result[0][1],result[0][5],result[0][9]))
    tensor_x2 = torch.stack((result[0][2],result[0][6],result[0][10]))
    tensor_y2 = torch.stack((result[0][3],result[0][7],result[0][11]))

    tensor_x1 = (tensor_x1 - norm_data_x) * -35
    tensor_y1 = (tensor_y1 - norm_data_y) * -35
    tensor_x2 = (tensor_x2 + norm_data_x) * 35
    tensor_y2 = (tensor_y2 + norm_data_y) * 35


print((time.time()-s)/1000)

0.00019782114028930664


In [29]:
score_tensor = result[1] >=0.3
output = []

s = time.time()
for i in range(1000) : 

    for label in range(3): 
        new_x1 = tensor_x1[label][score_tensor[label]]
        new_y1 = tensor_y1[label][score_tensor[label]]
        new_x2 = tensor_x2[label][score_tensor[label]]
        new_y2 = tensor_y2[label][score_tensor[label]]
print((time.time()-s)/1000)

0.0007558000087738037


In [57]:


s1 = time.time()
for i in range(1000) : 

    tensor_x1 = torch.stack((result[0][0],result[0][4],result[0][8]))
    tensor_y1 = torch.stack((result[0][1],result[0][5],result[0][9]))
    tensor_x2 = torch.stack((result[0][2],result[0][6],result[0][10]))
    tensor_y2 = torch.stack((result[0][3],result[0][7],result[0][11]))

    tensor_x1 = (tensor_x1 - norm_data_x) * -35
    tensor_y1 = (tensor_y1 - norm_data_y) * -35
    tensor_x2 = (tensor_x2 + norm_data_x) * 35
    tensor_y2 = (tensor_y2 + norm_data_y) * 35



    score_tensor = result[1] >=0.3
    output = []

    ori_w = 640
    ori_h = 426

    for label in range(3) : 

        new_x1 = tensor_x1[label][score_tensor[label]]
        new_y1 = tensor_y1[label][score_tensor[label]]
        new_x2 = tensor_x2[label][score_tensor[label]]
        new_y2 = tensor_y2[label][score_tensor[label]]


    #     box = torch.stack((new_x1,new_y1,new_x2,new_y2),1)

        box = torch.stack((new_x1,new_y1,new_x2,new_y2),1)


        score = result[1][label][score_tensor[label]]
        s2 = time.time()
        nms_out_index = torchvision.ops.nms(
            box,
            score,
            0.5,
        )
print((time.time()-s1)/1000)

#     result_box = box[nms_out_index].detach().cpu().numpy()
#     result_score = score[nms_out_index].detach().cpu().numpy()

#     for idx, bbox in enumerate(result_box):
#         if isinstance(bbox, type(None)):
#             output.append(None)
#             continue
#         x1, y1, x2, y2 = map(int, bbox)
#         x1 = int((x1 * ori_w)/input_w)
#         y1 = int((y1 * ori_h)/input_h)
#         x2 = int((x2 * ori_w)/input_w)
#         y2 = int((y2 * ori_h)/input_h)
#         score = str(result_score[idx])
#         output.append({"bbox":[x1, y1, x2, y2], "score":score, "label":label})



# img_path = '/DATA_17/ij/test/test_image.jpeg'
# image = cv2.imread(img_path)
# for result in output:
#     xmin, ymin, xmax, ymax = result['bbox']
#     color = [255, 0, 0] 
#     cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, 2)
# plt.imshow(image)
# plt.show()
                    

0.001633723020553589


In [ ]:
for i in range(3):
    print(i)

In [ ]:
t1 = torch.FloatTensor([[1., 2., 3.],
                       [4., 5., 6.],
                       [7., 8., 9.],
                       [10., 11., 12.]
                      ])
t2 = torch.FloatTensor([[11., 12., 13.],
                       [14., 15., 16.],
                       [17., 18., 19.],
                       [27., 21., 22.]
                      ])
s1 = time.time()
a = torch.stack((t1, t2))
print(a.shape, time.time()-s1)

s2 = time.time()
d = torch.cat([t1,t2], 0).view(-1,4,3) 
print(d.shape, time.time()-s2)


# c,d = torch.max(a, 1, keepdim=True)
# # b = a.ravel()
# # a[0,3,2]

# # c = a[0]
# # d = a[1]

# # d.shape
# print(c)
# print(d)
# print(c.shape)
# print(d.shape)

# b = a > 10 

# a[0][b[0]]

In [ ]:
a = [0.014285714285714285, 0.4714285714285714, 0.9285714285714286, 1.3857142857142857, 1.8428571428571427, 2.3, 2.757142857142857, 3.2142857142857144, 3.6714285714285713, 4.128571428571429, 4.585714285714285, 5.042857142857143, 5.5, 5.957142857142857, 6.414285714285715, 6.871428571428571, 7.328571428571428, 7.785714285714286, 8.242857142857142, 8.7, 9.157142857142857, 9.614285714285714, 10.071428571428571, 10.528571428571428, 10.985714285714286, 11.442857142857143, 11.9, 12.357142857142858, 12.814285714285715, 13.271428571428572, 13.728571428571428, 14.185714285714285, 14.642857142857142, 15.1, 15.557142857142857, 16.014285714285716, 16.47142857142857, 16.928571428571427, 17.385714285714286, 17.84285714285714, 18.3, 18.757142857142856, 19.214285714285715, 19.67142857142857, 20.12857142857143, 20.585714285714285, 21.042857142857144, 21.5, 21.957142857142856, 22.414285714285715, 22.87142857142857, 23.32857142857143, 23.785714285714285, 24.242857142857144, 24.7, 25.15714285714286, 25.614285714285714, 26.071428571428573, 26.52857142857143, 26.985714285714284]
b = [0.014285714285714285, 0.4714285714285714, 0.9285714285714286, 1.3857142857142857, 1.8428571428571427, 2.3, 2.757142857142857, 3.2142857142857144, 3.6714285714285713, 4.128571428571429, 4.585714285714285, 5.042857142857143, 5.5, 5.957142857142857, 6.414285714285715, 6.871428571428571, 7.328571428571428, 7.785714285714286, 8.242857142857142, 8.7, 9.157142857142857, 9.614285714285714, 10.071428571428571, 10.528571428571428, 10.985714285714286, 11.442857142857143, 11.9, 12.357142857142858, 12.814285714285715, 13.271428571428572, 13.728571428571428, 14.185714285714285, 14.642857142857142, 15.1]
len(a
   )

In [ ]:
a = [0.7430, 0.8202, 0.8331, 0.8958, 0.8933, 0.7453, 0.8693, 0.8905, 0.8900,
        0.8025, 0.8258, 0.8339, 0.6816, 0.8935, 0.8963, 0.8787, 0.8909, 0.8937,
        0.8951, 0.8220, 0.8422, 0.8935, 0.8947, 0.5792, 0.8929, 0.8936, 0.8319]

In [ ]:
b = torch.tensor(a)
b.shape

In [ ]:
t1 = torch.FloatTensor([[1., 2., 3.],
                       [4., 5., 6.],
                       [7., 8., 9.],
                       [10., 11., 12.]
                      ])
t2 = torch.FloatTensor([[11., 12., 13.],
                       [14., 15., 16.],
                       [17., 18., 19.],
                       [27., 21., 22.]
                      ])


d = torch.cat([t1,t2], 0).view(2,4,3) 
d.shape

In [ ]:
num_classes = list(range(3))
for i in num_classes : 
    print(i)

In [ ]:
img_path = '/DATA_17/ij/test/test_image.jpeg'
image = cv2.imread(img_path)

# image = cv2.imread(img_path)[..., ::-1]#BGR 순서를 RGB로 뒤집습니다.
image = cv2.imread(img_path)
image = np.copy(image)
image = torch.from_numpy(image).to(torch.device("cuda"))
image = image.permute(2,0,1)
image.shape

In [ ]:
a = torch.tensor([[1,2,3],
                  [4,5,6]])
b = torch.tensor([[10,20,30],
                  [40,50,60]])
c = torch.cat([a,b],dim=1)
e = torch.stack((a,b),1)
d = c.view([-1,2])
